In [107]:
import numpy as np


def distance(p1, p2):
    """Returns the distance between two points p1 and p2"""
    return np.sqrt(np.sum((p1 - p2)**2))

In [108]:
import scipy.stats as ss
import random


def majority_vote(votes):
    """Returns majority vote and its count in a list of votes"""
    vote_counts = {}
    for vote in votes:
        if vote in vote_counts:
            vote_counts[vote] += 1
        else:
            vote_counts[vote] = 1
    winners = []
    max_counts = max(vote_counts.keys())
    for vote, count in vote_counts.items():
        if count == max_counts:
            winners.append(vote)
    return random.choice(winners), max_counts


def majority_vote_short(votes):
    """Returns majority vote and its count in a list of votes"""
    mode, count = ss.mstats.mode(votes)
    return mode, count